# 基于大语言模型的题库生成器

### 1. 获取 API key 并创建调用 API 的函数

首先使用`pip install dashscope`安装阿里云百炼的 Python SDK

然后导入相关依赖包，使用`getpass`输入api_key

In [1]:
from getpass import getpass
from dashscope import Generation
import data, LLM_functions, interface_gradio, question_list, RAG_vector_store
import os
# 安全获取API Key（输入时不显示明文）
# api_key = getpass("请输入阿里云百炼API Key: ")
api_key = "sk-c64994ca9f9741888dc7bccdef2aa0a1"
os.environ["DASHSCOPE_API_KEY"] = api_key
Generation.api_key = api_key


### 2. 通过关键词检索知识库，并获取相应的文段

使用`pip install pypdf faiss-cpu`安装依赖包

#### 2.1 初步创建向量数据库，根据所选文件进行切分和向量化
#### 2.2 加载已有的向量数据库并添加新文档
#### 2.3 根据关键词检索数据库

In [2]:
### 完整RAG流程测试
# import RAG_vector_store

# RAG_vector_store.create_vector_store("./data/sample.pdf")
# RAG_vector_store.add_new_documents_to_vector_store("./data/alibaba-2.txt")
# results = RAG_vector_store.search_similar_documents("延长学制", k=5)
# results2 = RAG_vector_store.search_similar_documents("阿里", k=5)

# print("------------------------------")
# print(results)

In [ ]:
import LLM_functions
# 测试调用

keyword = "阿里"
result = LLM_functions.generate_QA(keyword=keyword, nums=5, model="qwen-plus", api_key=os.environ.get("DASHSCOPE_API_KEY"))

查询: 延长学制
找到 5 个相关文档:

--- 文档 1 ---
内容: — 13 —
阶段）；临床医学八年制本博连读生的博士研究生阶段基本修业
年限为4 年。
基本修业年限由各学科专业在本办法规定的年限内予以明
确。“强基计划”等本研贯通项目录取转段的研究生基本修业年
限要求依据相关项目培养规定确定。
第二十八条 研究生在基本修业年限内不能完成学业的，可
申请延长修业年限（以下简称延期）。全日制博士研究生最长修
业年限为在原基本修业年限基础上延长 2 年；全日制硕士研...
来源: ./data/sample.pdf
页码: 12
--------------------------------------------------
--- 文档 2 ---
内容: — 22 —
包括本数；所称的“不满”“超过”，不包括本数。
第五十六条 本办法由研究生院负责解释。
第五十七条 本办法自发布之日起施行，《浙江大学研究生
学籍管理实施办法》（浙大发研〔2017〕115 号）同时废止。本
办法实施之前已入学的研究生，可以继续适用原办法。
抄送：纪委，各院级党委，党委各部门，各党工委，工会、团委。
浙江大学校长办公室 主动公开 2024 年8 月16 日印发...
来源: ./data/sample.pdf
页码: 21
--------------------------------------------------
--- 文档 3 ---
内容: — 2 —
浙江大学研究生学籍管理实施办法
第一章 总 则
第一条 为全面贯彻国家教育方针，规范研究生学籍管理，
维护研究生正常教育教学秩序，保障研究生身心健康与合法权
益，促进研究生全面发展，依据教育部《普通高等学校学生管理
规定》（中华人民共和国教育部令第 41 号），制定本办法。
第二条 本办法适用于学校对接受普通高等学历教育的研
究生的管理。
第二章 新生入学
第三条 研究生新生应当在学校...
来源: ./data/sample.pdf
页码: 1
--------------------------------------------------
--- 文档 4 ---
内容: — 12 —
超过 12 个月，且计入最长修业年限。创业休学、生育休学、因
公出国（境）单次 6 个月以上研究生的修业年限以第

In [10]:
### 测试：根据题库答案生成干扰选项
print(result[3])
print(type(result[3]))

# 生成选择题
ans = LLM_functions.generate_options(
    answer=str(result[3]),
)
print(ans)
print(type(ans))

# 生成填空题
# LLM_functions.generate_fill_in_the_blank(
#     QA=str(result[3]),
# )

{'Q': '研究生新生因故不能按期入学，应如何办理请假手续？', 'A': '应当事先向学院（系）研究生管理部门书面请假，并附相关证明材料，经学院（系）分管领导批准后生效。'}
<class 'dict'>
应当提前向学校教务处口头请假，并提交相关证明材料，经教务处审批后生效。  
可以先暂缓入学，待情况稳定后再向研究生院提交补办入学手续申请。  
应直接联系校团委办理请假手续，并提供相关证明，由校团委负责人签字同意即可。
<class 'str'>


### 3. 将生成的题库保存到本地

In [5]:
import question_list

# 调用函数添加question_list
question_list.add_question_list(
    existing_file_path="./question_list/question_lists.json", 
    new_list_name=keyword,
    new_qa_list=result
)

已成功添加延长学制，与原有列表并列


### 4. 后端接口设计

- 导入本地文件
  - .txt
  - .md?
  - .pdf
- 根据主题检索知识库文段
- 文段导入大模型生成QA问答
  - 单个文段导入大模型
  - 循环生成
- 处理生成的QA问答实现数据格式化
- 保存生成的题库至本地


### 5. 使用 gradio 实现前端界面

In [ ]:
import interface_gradio

# 启动gradio界面
if __name__ == "__main__":
    # demo.launch(share=True)
    interface_gradio.create_main_interface().launch(share=False)


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Exception in thread Thread-9:
Traceback (most recent call last):
  File "d:\anaconda\envs\chatgpt\lib\site-packages\httpx\_transports\default.py", line 101, in map_httpcore_exceptions
    yield
  File "d:\anaconda\envs\chatgpt\lib\site-packages\httpx\_transports\default.py", line 250, in handle_request
    resp = self._pool.handle_request(req)
  File "d:\anaconda\envs\chatgpt\lib\site-packages\httpcore\_sync\connection_pool.py", line 256, in handle_request
    raise exc from None
  File "d:\anaconda\envs\chatgpt\lib\site-packages\httpcore\_sync\connection_pool.py", line 236, in handle_request
    response = connection.handle_request(
  File "d:\anaconda\envs\chatgpt\lib\site-packages\httpcore\_sync\http_proxy.py", line 316, in handle_request
    stream = stream.start_tls(**kwargs)
  File "d:\anaconda\envs\chatgpt\lib\site-packages\httpcore\_sync\http11.py", line 376, in start_tls
    return self._stream.start_tls(ssl_context, server_hostname, timeout)
  File "d:\anaconda\envs\chatgpt\l

查询: 基本年限
找到 1 个相关文档:

--- 文档 1 ---
内容: — 13 —
阶段）；临床医学八年制本博连读生的博士研究生阶段基本修业
年限为4 年。
基本修业年限由各学科专业在本办法规定的年限内予以明
确。“强基计划”等本研贯通项目录取转段的研究生基本修业年
限要求依据相关项目培养规定确定。
第二十八条 研究生在基本修业年限内不能完成学业的，可
申请延长修业年限（以下简称延期）。全日制博士研究生最长修
业年限为在原基本修业年限基础上延长 2 年；全日制硕士研...
来源: ./data/sample.pdf
页码: 12
--------------------------------------------------
已成功添加基本年限，与原有列表并列
